In [1]:
import os
import gc   # garbage collector
import time
import pandas as pd
import numpy as np

import rasterio
import rioxarray as rioxr

import dask_gateway
import dask.array as da

from joblib import load

import geopandas as gpd
from shapely.geometry import mapping

from scipy.ndimage import median_filter

# custom modules
import raster_to_features as rf
import A_data_sampling_workflow.sample_rasters as sr
import shutil

In [2]:
# year for which to predict iceplant locations
year = 2020

# whether to apply median filter to raster
filter_rasters = True

# whether to save rasters
save_rasters = True
prefix = 'snow13'

# whether to print processing info at runtime
verbose = True

save_processing_times = True

delete_aux_rasters = True

# whether only to process aois
only_aois = True

uses_ndvi = True


feature_order = ['r', 'r_avg13', 'r_entr13', 
                 'g', 'g_avg13', 'g_entr13', 
                 'b', 'b_avg13', 'b_entr13', 
                 'nir', 'nir_avg13', 'nir_entr13', 
                 'ndvi', 'ndvi_avg13','ndvi_entr13', 
                 'month', 'day_in_year']

# **************************************************************
# radius of the disk (in pixels) over which entropy is calculated
entropy_r = 6

filter_side = 5

In [3]:
# length of side of the square window over which average/max/min are calculated.
box_side = entropy_r *2 +1

# **************************************************************
# open shapefile of SB coastal buffer and process it to use it for clipping
fp = os.path.join(os.getcwd(), 
                  'separating_naip_flights', 
                  'SB_coastal_buffer', 
                  'SB_coastal_buffer.shp')
coast = gpd.read_file(fp)
coast_geo = coast.geometry.apply(mapping)

# **************************************************************
# load pre-trained random forest classifier
rfc = load(prefix+'_rfc.joblib') 

# **************************************************************

if only_aois:
    scene_ids = ['ca_m_3412037_nw_10_060_20200607',
                 'ca_m_3412039_nw_10_060_20200522',
                 'ca_m_3412040_ne_10_060_20200522',
                 'ca_m_3411934_sw_11_060_20200521',
                 'ca_m_3411936_se_11_060_20200521']
else:    
    # select the scene ids from given year that intersect the coastal buffer
    # the itemids of all scenes that intersect the coast were previously stored in a csv
    scene_ids = pd.read_csv(os.path.join(os.getcwd(),
                                         'separating_naip_flights',
                                         'coastal_scenes_ids.csv'))
    scene_ids = scene_ids.loc[scene_ids['year'] == year]
    scene_ids = scene_ids.reset_index().itemid

# **************************************************************
# prepare folder to save rasters
if save_rasters:
    fp = os.path.join(os.getcwd(), 'processing_results')
    if os.path.exists(fp) == False:
        os.mkdir(fp)
    if filter_rasters:
        fp = os.path.join(fp, prefix+'_filter_clip_preds_' + str(year))
    else:
        fp = os.path.join(fp, prefix+'_clip_preds_' + str(year))
    if os.path.exists(fp) == False:
        os.mkdir(fp)

In [4]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(15)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.768c132563f54d67a529a524162f3fa3/status,


In [ ]:
# ---------------------------------------
# collect processing information for each scene
times_pre = []
times_class = []
times_post = []
processed = []
reason = []
veg_pixels = [] # number of pixels with ndwi<0.3 and ndwi>0.05
n_pixels = []   # number of non-zero pixels in masked scene

# counter for scenes queued for processing
N = len(scene_ids)
t_total = time.time()

# ---------------------------------------
# ---------------------------------------

for itemid in scene_ids:    
    # ***********************************************************************************************
    # *************************************** PRE-PROCESSING ****************************************
    # open NAIP scene and clip to coast
    t0 = time.time()
    raster = rf.rioxr_from_itemid(itemid).rio.clip(coast_geo, coast.crs)

    # ---------------------------------------
    # select pixels with data (blacked out portions have 0 on all bands)
    df = rf.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])
    df = df.loc[ (df['nir'] != 0) | (df['r'] != 0) | (df['g'] != 0) | (df['b'] != 0)]
    n_pixels.append(df.shape[0])

    # ---------------------------------------
    # stop if there's no data at intersection
    if df.shape[0] == 0:
        rf.finish_processing('no_data', processed, reason, times_pre, times_class, times_post, veg_pixels, itemid)
        if verbose:
            rf.finish_processing_message('no_data', itemid)

    else:
        # find vegetation pixels to go into model
        # keep indices of water and low-ndvi pixels
        # add ndvi and ndwi features for each pixel
        if verbose:
            print('selected data on scene', time.time()-t0)
        t0 = time.time()
        is_veg, water_index, not_veg_index = rf.add_spectral_features(df, 
                                                                      ndwi_thresh = 0.3, 
                                                                      ndvi_thresh = 0.05) 
      # ---------------------------------------
        # stop if there are no vegetation pixels at intersection
        if is_veg.shape[0] == 0:
            rf.finish_processing('no_veg', processed, reason, times_pre, times_class, times_post, veg_pixels, itemid)            
            if verbose:
                rf.finish_processing_message('no_veg', itemid)

        else:
            processed.append('Y')
            reason.append('processed')
            if verbose:
                print('selected vegetation pixels', time.time()-t0)
            
            # ---------------------------------------
            # discard ndwi and add date features
            is_veg.drop('ndwi', axis=1, inplace=True)
            is_veg = rf.add_date_features(is_veg, rf.rioxr_from_itemid(itemid).datetime)


    # *************************************************************************************************
    # ******************************** CREATE R,G,B,NIR AUXILIARY RASTERS *****************************
            t0 = time.time()
            # make auxiliary spectral rasters from clipped NAIP 
            band_names = ['r_', 'g_', 'b_', 'nir_']
#            tags = ['_maxs', '_mins', '_avgs', '_entrs']
            tags = ['_avgs', '_entrs']
            window_fps = []
            window_cols = []

            for name, band in zip(band_names,range(1,5)):
                rast_name = name+itemid
                
                for tag in tags:
                    rast_fp = os.path.join(os.getcwd(), 'temp', rast_name + tag + '.tif')
                    window_fps.append(rast_fp)        
                    window_cols.append( name.replace('_','')+tag.replace('s',str(box_side)))
                    
                    if os.path.isfile(rast_fp) == False:
                        if tag == '_avgs':
                            sr.avg_raster(raster = raster, band=band, rast_name=rast_name, n=box_side)                            
                        elif tag == '_entrs':
                            sr.entropy_raster(raster=raster, band=band, rast_name=rast_name, n=entropy_r)
                
                    #sr.max_min_avg_rasters(raster = raster, band=band, rast_name=rast_name, n=box_side)

            if verbose:
                print('created/verified R,G,B,NIR auxiliary rasters (max, min, avgs, entr)', time.time()-t0)


    # ********************************************************************************************
    # ******************************** CREATE NDVI AUXILIARY RASTERS *****************************
            if uses_ndvi:
                t0 = time.time()
                # make auxiliary NDVI from clipped NAIP 
                ndvi = sr.ndvi_xarray(raster)
                band_names.append('ndvi_')
                rast_name = 'ndvi_'+itemid
                
                for tag in tags:
                    rast_fp = os.path.join(os.getcwd(), 'temp', rast_name + tag + '.tif')
                    window_fps.append(rast_fp)        
                    window_cols.append('ndvi' + tag.replace('s',str(box_side)))                
                    
                    if os.path.isfile(rast_fp) == False:
                        if tag == '_avgs':
                            sr.avg_raster(rast_data=ndvi, 
                                          crs=raster.rio.crs, 
                                          transf=raster.rio.transform(), 
                                          rast_name=rast_name, 
                                          n=box_side)
                        elif tag == '_entrs':
                            # adjusting to entropy input types
                            ndvi = ndvi*100+100  
                            sr.entropy_raster(rast_data=ndvi.astype('uint8'), 
                                              crs=raster.rio.crs, 
                                              transf=raster.rio.transform(), 
                                              rast_name=rast_name, 
                                              n=entropy_r)
                if verbose:
                    print('created/verified NDVI auxiliary rasters (max,min,avgs,entr)', time.time()-t0)
                #free memory
                del ndvi
                gc.collect()


    # *******************************************************************************************
    # *********************** EXTRACT FEATURES FROM AUXILIARY RASTERS ***************************
            window_values = []    
            for fp_aux in window_fps:
                match = rioxr.open_rasterio(fp_aux).squeeze()
                match_vector = match.to_numpy().reshape(match.shape[0]*match.shape[1])
                window_values.append(match_vector)
                if delete_aux_rasters:
                    os.remove(fp_aux)

            df_window = pd.DataFrame(dict(zip( window_cols, window_values)))

            scene_features = pd.concat([is_veg, df_window.iloc[is_veg.index]], axis=1)

            # **********************************************************************************************
            # *************** REMOVE NA VALUES (PIXELS AT EDGE OF CLIPPED PART OF RASTER) ******************
            # combine indices for r_min == 0 and ndvi_min == nan, ndvi_avg == nan

            # **** THIS NEEDS TO BE NOT HARDCODED: r_min11 and ndvi_min11 ****
            remove = set()
            if '_min' in tags:
                remove = scene_features.r_min11[scene_features.r_min11 == 0].index
            if uses_ndvi:
                for band in [x.replace('s',str(box_side)) for x in ['ndvi'+ y for y in tags]]:
                    remove = remove.union(scene_features[band][scene_features[band].isna() == True].index)
            # remove these indices from scene_features
            # no need to add them anywhere else, they will be part of the raster's background
            scene_features = scene_features.drop(remove)

            #free memory            
            del df_window, window_values, match_vector, match, remove
            gc.collect()  

            times_pre.append(time.time()-t0)

            # ******************************************************************************
            # ******************************** ORDER FEATURES ****************************** 

            scene_features = scene_features[feature_order]
            if verbose:
                print('finished assembling features')

            # ***********************************************************************************************
            # *************************************** CLASSIFICATION ****************************************
            # convert into dask.array and predict using model
            da_pixels = da.from_array(np.array(scene_features), chunks=728802)
            scene_preds = rfc.predict(da_pixels)

            # ---------------------------------------
            t0 = time.time()
            preds = scene_preds.compute()
            times_class.append(time.time() - t0)
            if verbose:
                print('finished classification')

            # ************************************************************************************************
            # *************************************** POST-PROCESSING ****************************************
            # recover pixel indices for iceplant classifications
            t0 = time.time()
            preds_df = pd.DataFrame(preds, 
                                 columns=['is_iceplant'], 
                                 index = scene_features.index)
            is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
            non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

            # ---------------------------------------
            # reconstruct indices into image
            indices = [non_iceplant_index,
                       is_iceplant_index, 
                       not_veg_index,
                       water_index]
            values = [0,    # values assigned to pixels from each index
                      1,
                      2,
                      3]
            reconstruct = rf.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)

            # ---------------------------------------
            # apply median 3x3 filter if needed
            if filter:
                reconstruct = median_filter(reconstruct, size=filter_side)

            times_post.append(time.time() - t0)
            if verbose:
                print('finished post-processing')

            # ************************************************************************************************
            # *************************************** SAVE RASTERS *******************************************  
            if save_rasters:
                if filter_rasters:
                    filename = prefix+'_filter_clip_preds_' + itemid + '.tif'
                else:
                    filename = prefix+'_clip_preds_' + itemid + '.tif'

                with rasterio.open(
                    os.path.join(fp, filename),  # file path
                    'w',           # w = write
                    driver = 'GTiff', # format
                    height = reconstruct.shape[0], 
                    width = reconstruct.shape[1],
                    count = 1,  # number of raster bands in the dataset
                    dtype = rasterio.uint8,
                    crs = raster.rio.crs,
                    transform = raster.rio.transform(),
                ) as dst:
                    dst.write(reconstruct.astype(rasterio.uint8), 1)

            if verbose:
                print('FINISHED: ', itemid)        

    # ***********************************************************************************************
    # ************************************ FINAL INFO MESSAGE ***************************************            
    N = N-1        
    if verbose:
        print('REMAINING: ', N, 'scenes', '\n')
    else:
        print('REMAINING: ', N, 'scenes', end="\r")
    
print('TOTAL TIME: ', (time.time() - t_total)/60, ' mins')

selected data on scene 21.84367561340332
selected vegetation pixels 3.366628885269165


In [ ]:
#save times processed and itemids as dataframe
D = { 'itemid': scene_ids,
     'processed': processed,
     'reason':reason,
     'pre_times': times_pre,
     'class_times' : times_class,
     'post_times' : times_post, 
     'processed_pix' : n_pixels }
processing_df = pd.DataFrame(D)

In [ ]:
if save_processing_times:
    fp = os.path.join(os.getcwd(),'processing_results')
    if os.path.exists(fp) == False:
        os.mkdir(fp)


    if filter_rasters:
        filename = prefix+'_filter_clip_processing_results_' + str(year) + '.csv'
    else:
        filename = prefix+'_clip_processing_results_' + str(year) + '.csv'

    processing_df.to_csv(os.path.join(fp, filename ), index=False)